In [1]:
from pathlib import Path
from datatree import DataTree
import dask

import pandas as pd
import valenspy as vp
import xarray as xr

ERA5 Land dataset zonder time bnds

In [2]:
# Function to preprocess files and remove 'time_bnds'
def preprocess(ds):
    return ds.drop_vars('time_bnds', errors='ignore')

# Load ERA 5 land data with preprocessing to drop 'time_bnds'
era5_land_files = list(Path("/mnt/HDS_BREGILABEPOC/BREGILABEPOC/bertvs/era5-land/belgium/daily").rglob("*.nc"))
era5_land_ds = xr.open_mfdataset(era5_land_files, chunks="auto", preprocess=preprocess)

In [3]:
era5_land_ds

<xarray.Dataset> Size: 7GB
Dimensions:    (time: 52596, longitude: 41, latitude: 25)
Coordinates:
  * time       (time) datetime64[ns] 421kB 1950-01-01T11:00:00 ... 2021-12-31...
  * longitude  (longitude) float32 164B 2.0 2.125 2.25 2.375 ... 6.75 6.875 7.0
  * latitude   (latitude) float32 100B 52.0 51.88 51.75 ... 49.25 49.12 49.0
Data variables: (12/18)
    d2m        (time, latitude, longitude) float64 431MB dask.array<chunksize=(730, 25, 41), meta=np.ndarray>
    e          (time, latitude, longitude) float64 431MB dask.array<chunksize=(730, 25, 41), meta=np.ndarray>
    pev        (time, latitude, longitude) float64 431MB dask.array<chunksize=(730, 25, 41), meta=np.ndarray>
    ro         (time, latitude, longitude) float64 431MB dask.array<chunksize=(730, 25, 41), meta=np.ndarray>
    sde        (time, latitude, longitude) float64 431MB dask.array<chunksize=(730, 25, 41), meta=np.ndarray>
    sf         (time, latitude, longitude) float64 431MB dask.array<chunksize=(730, 25, 41), meta=np.ndarray>
    ...         ...
    ssrd       (time, latitude, longitude) float64 431MB dask.array<chunksize=(730, 25, 41), meta=np.ndarray>
    swvl1      (time, latitude, longitude) float64 431MB dask.array<chunksize=(730, 25, 41), meta=np.ndarray>
    t2m        (time, latitude, longitude) float64 431MB dask.array<chunksize=(730, 25, 41), meta=np.ndarray>
    tp         (time, latitude, longitude) float64 431MB dask.array<chunksize=(730, 25, 41), meta=np.ndarray>
    u10        (time, latitude, longitude) float32 216MB dask.array<chunksize=(731, 25, 41), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 216MB dask.array<chunksize=(730, 25, 41), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 1.9.9 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Mon Sep 05 05:06:06 2022: cdo -z zip daymin /mnt/HDS_BREGIL...
    frequency:    day
    CDO:          Climate Data Operators version 1.9.9 (https://mpimet.mpg.de...

In [4]:
from valenspy.cf_checks import cf_status
cf_status(era5_land_ds)

The file is NOT ValEnsPy CF compliant.
0.00% of the variables are ValEnsPy CF compliant
Unknown to ValEnsPy: ['d2m', 'e', 'pev', 'ro', 'sde', 'sf', 'skt', 'slhf', 'sp', 'sro', 'sshf', 'ssr', 'ssrd', 'swvl1', 't2m', 'tp', 'u10', 'v10']


In [5]:
from valenspy.inputconverter_functions import EOBS_to_CF, ERA5_to_CF, ERA5Land_to_CF
ic = vp.InputConverter(ERA5Land_to_CF)
era5_land_ds=ic.convert_input(era5_land_ds)

The file is NOT ValEnsPy CF compliant.
66.67% of the variables are ValEnsPy CF compliant
ValEnsPy CF compliant: ['evspsbl', 'evspsblpot', 'mrro', 'prsn', 'ts', 'hfls', 'ps', 'mrros', 'hfss', 'rsds', 'tas', 'pr']
Unknown to ValEnsPy: ['d2m', 'sde', 'ssr', 'swvl1', 'u10', 'v10']
